# Spark SQL y vistas temporales

Spark SQL es el módulo de Apache Spark para trabajar con datos estructurados (esquema definido). Permite ejecutar consultas SQL o usar una API estilo SQL (como .select, .where, etc.) sobre DataFrames, tanto en lenguaje SQL como en APIs de alto nivel (PySpark, Scala, etc.). En este laboratorio, nos centraremos únicamente en consultas mediante lenguaje SQL y la creación de vistas temporales.

### Preparación de entorno y análisis de dataset

In [ ]:
from pyspark.sql import SparkSession

# Crear la sesión de Spark
spark = SparkSession.builder.appName("PySpark03").getOrCreate()

In [ ]:
# Cargar datos
df = spark.read.option("header", True).csv("../../data/taxi_zone_lookup.csv")

# Ver las primeras filas
df.show(5)

# Esquema del DataFrame
df.printSchema()

## Vistas temporales

Las vistas temporales (temp views) son una forma de registrar un DataFrame como una tabla SQL temporal en la sesión de Spark. Esto permite usar SQL sobre cualquier DataFrame que ya hayas cargado o transformado. Esto no guarda nada en disco, y solo vive mientras dure la sesión de Spark.

Existe una variante que no se borra al cerrar la sesión, sino que se comparte entre sesiones Spark:
```python
df.createGlobalTempView("nombre_global")
spark.sql("SELECT * FROM global_temp.nombre_global")
```

##### ¿Cuándo usar vistas temporales?

- Cuando vienes del mundo SQL y te es más natural escribir consultas.
- Cuando quieres dividir la lógica de procesamiento en etapas (puedes crear una vista para cada paso).
- Cuando compartes lógica entre scripts o notebooks y necesitas una "tabla virtual" común.
- Cuando haces prototipado rápido.

In [ ]:
# Crear vista temporal
df.createOrReplaceTempView("taxi_zones")

## Funciones integradas y expresiones SQL

PySpark te permite realizar consultas SQL directamente sobre las tablas creadas. Por ejemplo, si tratáramos de responder a la pregunta "¿Cuántas zonas hay por borough?", podríamos formular la siguiente query:
```sql SELECT Borough, COUNT(DISTINCT Zone) AS num_zones
FROM taxi_zones
GROUP BY Borough
ORDER BY num_zones DESC 

Con Spark, podemos lanzarla directamente como una query de  SQL sin tener que traducirla a PySpark.

In [ ]:
spark.sql("""
    SELECT Borough, COUNT(DISTINCT Zone) AS num_zones
    FROM taxi_zones
    GROUP BY Borough
    ORDER BY num_zones DESC
""").show()

Ejemplo 2: "¿Cuántos LocationID hay por zona de servicio?"

In [ ]:
spark.sql("""
    SELECT service_zone, COUNT(*) AS location_count
    FROM taxi_zones
    GROUP BY service_zone
""").show()

Spark permite aplicar funciones como: AVG, COUNT, SUM, DATEDIFF, CASE WHEN, ROUND, etc.

#### Función _.explain()_

La función .explain() muestra el plan de ejecución de un DataFrame o consulta SQL.
Este plan es como un "manual de instrucciones" que Spark sigue para ejecutar tu transformación o query.

¿Para qué sirve?
- Ver cómo Spark transforma tus operaciones lógicas en físicas.
- Detectar si tus filtros y joins se están aplicando de forma eficiente.
- Saber si Spark está usando el Catalyst Optimizer y el Tungsten engine correctamente.
- Ayuda a detectar cuellos de botella o reprocesamientos innecesarios

In [ ]:
spark.sql("""
    SELECT service_zone, COUNT(*) AS location_count
    FROM taxi_zones
    GROUP BY service_zone
""").explain(True)

Por defecto _.explain()_ muestra el plan lógico y físico optimizado.

| Plan                       | ¿Qué muestra?                                                   |
| -------------------------- | --------------------------------------------------------------- |
| **Parsed Logical Plan**    | Tu código interpretado como operaciones lógicas                 |
| **Analyzed Logical Plan**  | Añade tipos de datos, validación de columnas                    |
| **Optimized Logical Plan** | Aplicación del Catalyst Optimizer (filtros, proyecciones, etc.) |
| **Physical Plan**          | Lo que Spark realmente ejecuta (por etapas y workers)           |


##### _.explain(mode="...")_ para más control
```python
df.explain(mode="formatted")
```
Modos disponibles:
- "simple": (por defecto) solo el plan físico
- "extended": todos los niveles (parsed, analyzed, optimized, physical)
- "codegen": código Java generado por Spark (útil para perf tuning avanzado)
- "cost": incluye costos estimados de cada etapa (experimental)
- "formatted": versión tabulada y más legible del extended

In [ ]:
spark.sql("""
    SELECT service_zone, COUNT(*) AS location_count
    FROM taxi_zones
    GROUP BY service_zone
""").explain(mode="cost")

### Join con Spark SQL

In [ ]:
# Cargar el dataset de viajes
trips_df = spark.read.parquet("../../data/yellow_tripdata_2023-01.parquet")

# Mostrar algunas filas
trips_df.show(5)

# Ver el esquema
trips_df.printSchema()

¿Cuáles son las zonas con más viajes iniciados? 

In [ ]:
trips_df.createOrReplaceTempView("yellow_trips")

In [ ]:
spark.sql("""
    SELECT z.Zone AS pickup_zone, COUNT(*) AS num_trips
    FROM yellow_trips t
    JOIN taxi_zones z
    ON t.PULocationID = z.LocationID
    GROUP BY z.Zone
    ORDER BY num_trips DESC
    LIMIT 10
""").show()

Detenemos la sesión de spark al finalizar

In [12]:
spark.stop()